<a href="https://colab.research.google.com/github/arkincognito/PyTorch/blob/main/04_Pytorch_Minibatch_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as func
import torch.nn as nn

# Define training dataset
>```
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
```



# Weights and Biases

Hypothesis:
$ H(x)= x\times W + b = \begin{pmatrix}
x_1 & x_2 & x_3 & \cdots & x_n\\
\end{pmatrix}
\times
\begin{pmatrix}
w_1\\
w_2\\
w_3\\
\vdots\\
w_n
\end{pmatrix} + b$
```
W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
hypothesis = x_train.mm(W) + b ## or x_train.matmul(W) + b
```

requires_grad indicates that the parameter will be trained.

#Optimizer and Learning Rate
We'll use SGD and Learning Rate of 0.01.
```
optimizer = torch.optim.SGD([W, b], lr = 0.01)
```

# Cost(MSE)

$
cost(W, b) = {1\over m}\sum \limits _{i=1} ^m (H(x^{(i)}) - y^{(i)})^2
$

```
cost = torch.mean((hypothesis - y_train)**2)
```

# Update

$\Delta w_j = {\partial cost \over {\partial w_j}}
= {2\over m}\sum \limits _{i=1} ^m (H(x^{(i)})x_j - y^{(i)})$<br>

$w_j : = w_j - lr \Delta w_j$

$\Delta b = {\partial cost \over {\partial b}}
= {2\over m}\sum \limits _{i=1} ^m (H(x^{(i)}) - y^{(i)})$<br>

$b : = b - lr \Delta b$


Initialize all the gradients to zero:
```
optimizer.zero_grad()
```

Backward Propagation:

```
cost.backward()
```

Update:
```
optimizer.step()
```


# nn.Module implementation

Instead of designing model from the scratch everytime, we could inherit the nn.Module class to build our model.<br>
Set the input and output for the linear layer.<br>
Write ```forward()```.<br>
```backward()``` will automatically propagate from cost.

```
class MultivariateLinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(3, 1) ## Designate input and output dimensions.

    def forward(self, x):
        return self.linear(x)
```

# nn.functional implementation

We can use predefined cost function provided in nn.functional.<br>
Changing cost function becomes much easier this way.

Instead of explicitly writing MSE Loss,
```
cost = torch.mean((hypothesis - y_train)**2)
```
implement the mse_loss method provided in nn.functional.
```
cost = torch.nn.functional.mse_loss(prediction, y_train)
```

# Minibatch
Inherit ```torch.utils.data``` to build dataset.
```
class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                    [93, 88, 93],
                    [89, 91, 90],
                    [96, 98, 100],
                    [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]
 
  def __len__(self):
    return len(self.x_data)
 
  def __get_item__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y
 
dataset=CustomDataset()
```

Feed this dataset into ```DataLoader``` to get batches of the dataset.

```
dataloader = DataLoader(dataset, batch_size=2, shuffle=True,)
```

```batch_size``` designates the size of each minibatch. Usually a power of 2.<br>
When ```shuffle``` is ```True```, shuffle batch data on each epoch.

In [ ]:
# Build dataset
class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                    [93, 88, 93],
                    [89, 91, 90],
                    [96, 98, 100],
                    [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]
 
  def __len__(self):
    return len(self.x_data)
 
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

# Build 1 layer linear regression model
class MultivariateLinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(3, 1) ## Designate input and output dimensions.

    def forward(self, x):
        return self.linear(x)
 
dataset=CustomDataset()

# Take a look at the dataset
x, y = dataset.__getitem__(slice(0,len(dataset)))
print(f'Train data x: {x.squeeze()}')
print(f'Train data y: {y.squeeze()}')

# Feed dataset through dataloader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True,)

# Build model feed parameters to update to SGD optimizer
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epoch = 20
for epoch in range(nb_epoch):
  # Load data in minibatches
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples
    # Calculate H(x)
    prediction = model(x_train)
    # Calculate cost
    cost = torch.nn.functional.mse_loss(prediction, y_train)
    # Update weights and bias
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # Show progress
    print('Epoch: {:4d}/{}\tBatch{}/{}\tcost: {}'.
          format(epoch+1, nb_epoch, batch_idx+1, len(dataloader), cost.item()))
print('train finished')

Train data x: tensor([[ 73.,  80.,  75.],
        [ 93.,  88.,  93.],
        [ 89.,  91.,  90.],
        [ 96.,  98., 100.],
        [ 73.,  66.,  70.]])
Train data y: tensor([152., 185., 180., 196., 142.])
Epoch:    1/20	Batch1/3	cost: 23864.25
Epoch:    1/20	Batch2/3	cost: 7236.0439453125
Epoch:    1/20	Batch3/3	cost: 1068.4761962890625
Epoch:    2/20	Batch1/3	cost: 730.6547241210938
Epoch:    2/20	Batch2/3	cost: 368.2947998046875
Epoch:    2/20	Batch3/3	cost: 97.2129898071289
Epoch:    3/20	Batch1/3	cost: 23.290531158447266
Epoch:    3/20	Batch2/3	cost: 5.450342178344727
Epoch:    3/20	Batch3/3	cost: 8.140374183654785
Epoch:    4/20	Batch1/3	cost: 1.7445882558822632
Epoch:    4/20	Batch2/3	cost: 0.1987265795469284
Epoch:    4/20	Batch3/3	cost: 0.06881655752658844
Epoch:    5/20	Batch1/3	cost: 1.171057939529419
Epoch:    5/20	Batch2/3	cost: 0.3484415113925934
Epoch:    5/20	Batch3/3	cost: 0.11361392587423325
Epoch:    6/20	Batch1/3	cost: 0.01632852852344513
Epoch:    6/20	Batch2/3	c

In [ ]:
x_test = torch.FloatTensor([72, 81, 76]).unsqueeze(0)
w, b = model.parameters()
print(f'w: {w},\nb:{b}')
print(f'y_test: {model(x_test).item()}')

w: Parameter containing:
tensor([[1.1188, 0.3812, 0.5132]], requires_grad=True),
b:Parameter containing:
tensor([-0.1042], requires_grad=True)
y_test: 150.3267364501953
